# ANLP Assignment 4

## Question 1 <br/>
### a.	Encode the FSA in terms of matrices, including initial and final states. (This is the transition matrix!) <br/>

State A

|    | S0 | S1 | S2 | S3 |
|----|----|----|----|----|
| S0 | 0  | 0  | 0  | 0  |
| S1 | 0  | 0  | 0  | 1  |
| S2 | 0  | 0  | 1  | 0  |
| S3 | 0  | 0  | 0  | 1  |


State B

|    | S0 | S1 | S2 | S3 |
|----|----|----|----|----|
| S0 | 0  | 1  | 0  | 0  |
| S1 | 0  | 1  | 0  | 1  |
| S2 | 0  | 0  | 1  | 0  |
| S3 | 0  | 1  | 0  | 1  |


State C

|    | S0 | S1 | S2 | S3 |
|----|----|----|----|----|
| S0 | 0  | 0  | 1  | 0  |
| S1 | 0  | 0  | 0  | 0  |
| S2 | 0  | 0  | 0  | 0  |
| S3 | 0  | 0  | 0  | 0  |


Initial State

| S0 | S1 | S2 | S3 |
|----|----|----|----|
| 1  | 0  | 0  | 0  |

Final State

| Final State| Values 
|-------------:|---|
| S0            | 1 | 
| S1            | 1 | 
| S2            | 1 |  
| S3            | 0 |




## b.	Describe the language that is accepted by the FSA as a regular expression <br/>

We will be using the state elimination method to obtain the regular expression: <br/>

The initial state and final state are in s0, hence we need to account for NULL values <br/>

Output for s2 can only be ca*<br/>

    There are two possile output for s1 one via s0 emitting b, the outcomes for that scenario are 1)bb* & 2)(b*a+b)* and the other scenario is s0 emitting a and the outcome for that will already be captured by (b*a+b)*

    So the Regular expression by combining them : (ca*)|(b|a+b)*
    
The null outcome is captured by the second term which can accept null values also 


## Question 2 : Markov Chain

(a) What is the probability that after 4 steps exactly 3 lines are busy? <br/>

The probability that 0 lines, 1 line, 2 lines and 3 lines are busy is <br/>

=$vP^4$ <br/>

= [0.18991 , 0.333, 0.2732, 0.20484] <br/>

So the probability that 3 lines will be busy is 0.2048 <br/> <br/>

(b) What number of lines being busy has the highest probability after 4 steps? <br/>

The highest probability corresponds to one line being busy with a probability of 0.33 <br/>


In [1]:
from nltk.corpus import brown
from nltk.util import ngrams
import numpy as np
import pandas as pd
import itertools
from collections import Counter,defaultdict

In [2]:
##The below codes were taken from prof. Damir Cavar's Git Repo "Part-of-Speech_Tagging"

##Calculating the probablity for tags and words based on Brown corpus 
tokens, tags = zip(*brown.tagged_words())
tagCounter = Counter(tags)
tokenCounter = Counter(tokens)
tokenTags = defaultdict(Counter)
for token, tag in brown.tagged_words():
    tokenTags[token][tag] +=1
tagTags = defaultdict(Counter)
posBigrams = list(ngrams(tags, 2))
for tag1, tag2 in posBigrams:
    tagTags[tag1][tag2] += 1
offset = 0
initialTags = Counter()
for x in brown.sents():
    initTag = tags[offset]
    initialTags[initTag] += 1
    offset += len(x)

##The above code segment was taken from prof. Damir Cavar's Git Repo "Part-of-Speech_Tagging"


$$P(t_1 \dots t_5\ |\ w_1 \dots w_5) = P(t_1|S)\ P(w_1|t_1)\ P(t_2|t_1)\ P(w_2|t_2)\ P(t_3|t_2)\ P(w_3|t_3)\ P(t_4|t_3)\ P(w_3|t_3)\ P(t_5|t_4)\ P(w_4|t_4)\ P(E|t_4)\ P(w_5|t_5)$$

In [3]:
##Word 
word=['time','flies','like','an','arrow']
total = len(tags)
dis_token=[]

##Identifying all possible tag combination
for i in range(len(word)):
    x=word[i]
    a=(tokenTags[x])
    p=list(a.keys())
    dis_token.append(p)
flat_list = [item for sublist in dis_token for item in sublist]
dis_token=np.unique(flat_list,axis=0)
list_for_bits = list(itertools.product(dis_token, repeat=5))
list_for_bits=pd.DataFrame(list_for_bits)

##Calculating the initial probablity
p=[]
for i in list(flat_list): 
    p.append([i,initialTags[i]/float(total)])
    
p=pd.DataFrame(p)
list_for_bits.columns = ['col1', 'col2','col3', 'col4','col5']
p.columns=['col1','Sent Start']

s1 = pd.merge(list_for_bits, p, how='left', on=['col1'])
p=[]
for i in list(flat_list): 
    p.append([i,tokenTags["time"][i]/tagCounter[i]])
 

##Calculating the probablity for the first word
p=pd.DataFrame(p)
p.columns=['col1','p for first token']
s2 = pd.merge(s1, p, how='left', on=['col1'])
s2['First value']=s2['Sent Start']*s2['p for first token']
s3=s2.loc[s2["First value"]>0,]

p=[]
for i in list(flat_list):
    for j in list(flat_list):
        p.append([i,j,tagTags[i][j]/tagCounter[i]])

p=pd.DataFrame(p)

In [4]:
## Calculating the transition probablity for different tags
p.columns=['col1','col2','Transition 1 to 2']
s3 = pd.merge(s3, p, how='left', on=['col1','col2'])

p.columns=['col2','col3','Transition 2 to 3']
s3 = pd.merge(s3, p, how='left', on=['col2','col3'])

p.columns=['col3','col4','Transition 3 to 4']
s3 = pd.merge(s3, p, how='left', on=['col3','col4'])

p.columns=['col4','col5','Transition 4 to 5']
s3 = pd.merge(s3, p, how='left', on=['col4','col5'])



In [5]:
##Calculating the probablity for each word 
p=[]
for i in list(flat_list): 
    p.append([i,tokenTags["flies"][i]/tagCounter[i]])
    
p=pd.DataFrame(p)
p.columns=['col2','p for 2nd token']
s3 = pd.merge(s3, p, how='left', on=['col2'])

p=[]
for i in list(flat_list): 
    p.append([i,tokenTags["like"][i]/tagCounter[i]])
    
p=pd.DataFrame(p)
p.columns=['col3','p for 3rd token']
s3 = pd.merge(s3, p, how='left', on=['col3'])


p=[]
for i in list(flat_list): 
    p.append([i,tokenTags["an"][i]/tagCounter[i]])
    
p=pd.DataFrame(p)
p.columns=['col4','p for 4th token']
s3 = pd.merge(s3, p, how='left', on=['col4'])


p=[]
for i in list(flat_list): 
    p.append([i,tokenTags["arrow"][i]/tagCounter[i]])
    
p=pd.DataFrame(p)
p.columns=['col5','p for last token']
s3 = pd.merge(s3, p, how='left', on=['col5'])

In [6]:
##Calculating the net probablity
s3['final prob']=s3['Sent Start']*s3['p for 2nd token']*s3['p for 3rd token']*s3['p for 4th token']* s3['p for last token']*s3['Transition 4 to 5']*s3['Transition 3 to 4']*s3['Transition 2 to 3']*s3['Transition 1 to 2']

In [7]:
s4=s3.drop_duplicates()

The maximum probablity is coming for the combination of "NN-HL,VBZ-HL,CS,AT and NN"

In [8]:
##Printing the final output, the first row contains the combination of POS with maximum probablity
s_final = s4.sort_values('final prob',ascending=False)
print(s_final)

          col1    col2    col3    col4    col5  Sent Start  p for first token  \
4594304  NN-HL  VBZ-HL      CS      AT      NN    0.000372           0.005438   
3834328     NN     VBZ      CS      AT      NN    0.001186           0.010146   
2421448     NN     NNS      CS      AT      NN    0.001186           0.010146   
4602886  NN-HL  VBZ-HL      VB      AT      NN    0.000372           0.005438   
7030969     VB     NNS      CS      AT      NN    0.000718           0.000030   
4576643  NN-HL     VBZ      CS      AT      NN    0.000372           0.005438   
2490104     NN     NNS      VB      AT      NN    0.001186           0.010146   
2426320     NN     NNS      IN      AT      NN    0.001186           0.010146   
3839200     NN     VBZ      IN      AT      NN    0.001186           0.010146   
7099625     VB     NNS      VB      AT      NN    0.000718           0.000030   
4400033  NN-HL     NNS      CS      AT      NN    0.000372           0.005438   
7035841     VB     NNS      

The maximum probablity is coming for the combination of "NN-HL,VBZ-HL,CS,AT and NN" tags

## Reference

https://github.com/dcavar/python-tutorial-for-ipython/blob/master/notebooks/Python%20Tutorial%20PoS%20Tagging.ipynb

# Question 4

The research paper discusses how the Maximum Entropy Model could be used for POS tagging and how it is an improvement over other methods like Markov Models, SDT and TBL. The paper also talks about problems in the corpus data and how we can overcome them.

The Maximum Entropy models uses a Bayesian approach. The algorithm splits the data into words and for each word it identifies the POS tag that maximizes the net entropy by, looking at the probability of various tags for that word, the probability of various tags occurring before the considered word and the words around it. The author doesn’t use any smoothing techniques and removes features below a particular threshold(features which occur less that 1000 times are removed). 

The author used the Beam Search method to identify best combination of POS tags(Beam value considered =5), this algorithm moves  left to right and identifies POS tags for each word.It considers the top N combination of POS tags based on Entropy at any point and it will take the best POS tag at the end. The author has also identified the words that are commonly mistaken by using the development set and has created special features for these words, this treatment with special features only resulted in a marginal increase in accuracy for the test set. 

The author considers the Maximum Entropy method over the Markov model since the Markov model considers only words previous to the current word whereas the Maximum entropy method could any useful information from the input data for tagging. Maximum entroy method doesnot require a lot of pre processing steps like the SDT algorithm, hence it is much easier to implement and maintain. Also the author considers the Maximum Entropy better that rule based algorithms like Transformation Based Learning since the latter doesn't provide probablity scores. 
